In [19]:
# y_seq_len + merged_future_prediction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
import os
import yfinance as yf
import timeit
import datetime
import calendar
import warnings
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from torch.nn.modules.transformer import TransformerDecoder, TransformerDecoderLayer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


warnings.filterwarnings("ignore", category=FutureWarning)


def create_sequences(data, seq_len):
    X = []
    y = []
    data = data.values  # This line is added
    for i in range(seq_len, data.shape[0]):
        X.append(data[i-seq_len:i, :])
        y.append(data[i:i+1, :])  # Change target shape to (1, n_features)

    X = np.array(X)
    y = np.array(y)
    return X, y

# def calculate_indicators(data, interval=None, seq_len=None):
#     # Adjust parameters based on frequency
#     if interval == '1wk':  # Weekly data
#         period = 6
#         rsi_period = 9
#         short_ema_period = 8
#         long_ema_period = 16
#         signal_period = 5
#         vol_period = 14
#         bband_window_size = 12
#     elif interval == '1mo':  # Monthly data
#         period = 4
#         rsi_period = 7
#         short_ema_period = 6
#         long_ema_period = 12
#         signal_period = 4
#         vol_period = 10
#         bband_window_size = 8
#     elif interval == '1d':  # Monthly data
#         period = 10
#         rsi_period = 14
#         short_ema_period = 12
#         long_ema_period = 26
#         signal_period = 9
#         vol_period = 26
#         bband_window_size = 20
#     elif interval == None:  # not specified
#         period = 5
#         rsi_period = 7
#         short_ema_period = 6
#         long_ema_period = 13
#         signal_period = 4
#         vol_period = 13
#         bband_window_size = 10

#     if data.shape[0] < long_ema_period + seq_len:
#         return None

#     # Check if 'Close' column exists in the data
#     if 'Close' in data.columns:
#         # Calculate daily returns
#         data['Return'] = data['Close'].pct_change()
#         # Calculate VPT (Volume-Price Trend)
#         data['VPT'] = (data['Volume'] * data['Return']).cumsum() 

#         # Calculate the percentage change for the price range
#         data['Price Range %'] = ((data['High'] - data['Low']) / data['Close']) * 100
#         # Calculate the percentage change for the candle body size
#         data['Body Size %'] = ((data['Open'] - data['Close']) / data['Close']) * 100
#         # Calculate the percentage change for the upper shadow size
#         data['Upper Shadow %'] = data.apply(lambda row: (row.High - max(row.Open, row.Close)) / row.Close * 100, axis=1)
#         # Calculate the percentage change for the lower shadow size
#         data['Lower Shadow %'] = data.apply(lambda row: (min(row.Open, row.Close) - row.Low) / row.Close * 100, axis=1)

#         lag = (period - 1) // 2
#         # Calculate adjusted price
#         adjusted_price = 2 * data['Close'] - data['Close'].shift(lag)

#         # Calculate ZLEMA directly using adjusted price
#         zlema = adjusted_price.ewm(span=period).mean()
#         data['ZLEMA'] = zlema

#         data['ZLEMA %'] = zlema.pct_change()

#         # Define tolerance as a percentage of the close price
#         tolerance = 0.015

#         # Initialize column with 0s
#         data['ZLEMA_Signal'] = 2

#         # Buy signal when Close is above (1 + tolerance) * ZLEMA
#         data.loc[data['Close'] > data['ZLEMA'] * (1 + tolerance), 'ZLEMA_Signal'] = 3

#         # Sell signal when Close is below (1 - tolerance) * ZLEMA
#         data.loc[data['Close'] < data['ZLEMA'] * (1 - tolerance), 'ZLEMA_Signal'] = 1

#         # Calculate the first exponential moving average (EMA1).
#         ema1 = data['Close'].ewm(span=period, min_periods=period).mean()
#         # Calculate the second exponential moving average (EMA2).
#         ema2 = ema1.ewm(span=period, min_periods=period).mean()
#         # Calculate the third exponential moving average (EMA3).
#         ema3 = ema2.ewm(span=period, min_periods=period).mean()
#         # Calculate the TEMA.
#         tema = 3 * ema1 - 3 * ema2 + ema3

#         # # Convert the TEMA to a Pandas Series.
#         data['TEMA'] = tema

#         data['TEMA %'] = tema.pct_change()

#         # Initialize column with 0s
#         data['TEMA_Signal'] = 2

#         # Buy signal when Close is above (1 + tolerance) * TEMA
#         data.loc[data ['Close']> data ['TEMA']* (1 + tolerance), 'TEMA_Signal']= 3

#         # Sell signal when Close is below (1 - tolerance) * TEMA
#         data.loc[data ['Close']< data ['TEMA']*(1-tolerance), 'TEMA_Signal']= 1

#         # Calculate RSI and its signal
#         delta = data['Close'].diff()
#         up, down = delta.copy(), delta.copy()

#         up[up < 0] = 0
#         down[down > 0] = 0

#         average_gain = up.rolling(window=rsi_period).mean()
#         average_loss = abs(down.rolling(window=rsi_period).mean())

#         rs = average_gain / average_loss

#         data['RSI'] =100 - (100 / (1 + rs))
#         data['RSI_Diff'] = data['RSI'].diff()

#         data["RSI_Signal"] =2
#         data.loc[data.RSI < 30,"RSI_Signal"] =3
#         data.loc[data.RSI > 70,"RSI_Signal"] =1

#         # Calculate MACD Line: (12-day EMA - 26-day EMA)
#         EMA_short = data['Close'].ewm(span=short_ema_period).mean()
#         EMA_long = data['Close'].ewm(span=long_ema_period).mean()
#         data['MACD_Line'] = EMA_short - EMA_long

#         # Calculate Signal Line: a n-day MA of MACD Line
#         data['Signal_Line'] = data["MACD_Line"].ewm(span=signal_period).mean()

#         # Generate MACD signals based on crossovers
#         data["MACD_Signal"] = 2 # Neutral
#         data.loc[(data.MACD_Line > data.Signal_Line) & (data.MACD_Line.shift() < data.Signal_Line.shift()),"MACD_Signal"] = 3 # Buy
#         data.loc[(data.MACD_Line < data.Signal_Line) & (data.MACD_Line.shift() > data.Signal_Line.shift()),"MACD_Signal"] = 1 # Sell

#         # Generate EMA signals based on crossovers
#         data["EMA_Short"]=EMA_short
#         data["EMA_Long"]=EMA_long

#         # Define a small percentage as a threshold
#         ema_diff_threshold = 0.01 # 1% difference

#         # Calculate absolute percent difference between short and long EMA
#         data['EMA_Diff'] = (EMA_short - EMA_long)/ EMA_long
#         data['EMA_Diff %'] = data['EMA_Diff'].pct_change()

#         data["EMA_Signal"] = np.where((data["EMA_Short"] > data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 3, 
#                              np.where((data["EMA_Short"] < data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 1, 2))

#         # Calculate Volatility as rolling standard deviation of log returns
#         data["Log_Return"] =(np.log(data.Close).diff())
#         data["Volatility"] =(data.Log_Return.rolling(window=vol_period, center=False).std())
#         data['Volatility %'] = data['Volatility'].pct_change()

#         # Generate Volatility signals based on high or low volatility
#         high_vol_threshold = data["Volatility"].quantile(0.85)
#         low_vol_threshold = data["Volatility"].quantile(0.15)
        
#         data['Volatility_Signal'] = 2
#         data.loc[data.Volatility > high_vol_threshold, 'Volatility_Signal'] = 3
#         data.loc[data.Volatility < low_vol_threshold, 'Volatility_Signal'] = 1

#         # Calculate Bollinger Bands
#         data["Middle_Band"] = data["Close"].rolling(window=bband_window_size, center=False).mean()
#         data["Std_Dev"] = data["Close"].rolling(window=bband_window_size, center=False).std()
#         data["Upper_Band"] = data["Middle_Band"] + data["Std_Dev"] * 2
#         data["Lower_Band"] = data["Middle_Band"] - data["Std_Dev"] * 2

#         # Identify shifting points
#         data["BBand_Signal"] = 2  # neutral

#         # Create a copy of the "Close" column
#         close_shifted = data["Close"].copy()

#         # Check for crossing Upper Band
#         cross_upper_band = (close_shifted.shift(1) <= data["Upper_Band"].shift(1)) & (data["Close"] > data["Upper_Band"])
#         data.loc[cross_upper_band, "BBand_Signal"] = 3  # buy

#         # Check for crossing Lower Band
#         cross_lower_band = (close_shifted.shift(1) >= data["Lower_Band"].shift(1)) & (data["Close"] < data["Lower_Band"])
#         data.loc[cross_lower_band, "BBand_Signal"] = 1  # sell

#         # Drop unnecessary columns
#         data.drop(["Middle_Band", "Std_Dev", "Open", 'Low', 'High', 'Volume'], axis=1, inplace=True)
#         data.dropna(inplace=True)

#     else:
#         print("'Close' column is not present in the input dataframe.")

#     return data

def calculate_indicators(data, interval=None, seq_len=None):
    # Adjust parameters based on frequency
    if interval == '1wk':  # Weekly data
        period = 6
        rsi_period = 9
        short_ema_period = 8
        long_ema_period = 16
        signal_period = 5
        vol_period = 14
        bband_window_size = 12
    elif interval == '1mo':  # Monthly data
        period = 4
        rsi_period = 7
        short_ema_period = 6
        long_ema_period = 12
        signal_period = 4
        vol_period = 10
        bband_window_size = 8
    elif interval == '1d':  # Monthly data
        period = 10
        rsi_period = 14
        short_ema_period = 12
        long_ema_period = 26
        signal_period = 9
        vol_period = 26
        bband_window_size = 20
    elif interval == None:  # not specified
        period = 5
        rsi_period = 7
        short_ema_period = 6
        long_ema_period = 13
        signal_period = 4
        vol_period = 13
        bband_window_size = 10
    if data.shape[0] < long_ema_period + seq_len:
        return None

    # Check if 'Close' column exists in the data
    if 'Close' in data.columns:
        # Calculate daily returns
        data['Return'] = data['Close'].pct_change()
        # Calculate VPT (Volume-Price Trend)
        data['VPT'] = (data['Volume'] * data['Return']).cumsum()
        # Price Range
        data['Price Range'] = data['High'] - data['Low']
        # Candle Body Size
        data['Body Size'] = data['Open'] - data['Close']
        # Upper Shadow Size
        data['Upper Shadow'] = data.apply(lambda row: row.High - max(row.Open, row.Close), axis=1)
        # Lower Shadow Size
        data['Lower Shadow'] = data.apply(lambda row: min(row.Open, row.Close) - row.Low, axis=1)

        lag = (period - 1) // 2
        # Calculate adjusted price
        adjusted_price = 2 * data['Close'] - data['Close'].shift(lag)

        # Calculate ZLEMA directly using adjusted price
        zlema = adjusted_price.ewm(span=period).mean()
        data['ZLEMA'] = zlema

        # Define tolerance as a percentage of the close price
        tolerance = 0.02

        # Initialize column with 0s
        data['ZLEMA_Signal'] = 2

        # Buy signal when Close is above (1 + tolerance) * ZLEMA
        data.loc[data['Close'] > data['ZLEMA'] * (1 + tolerance), 'ZLEMA_Signal'] = 3

        # Sell signal when Close is below (1 - tolerance) * ZLEMA
        data.loc[data['Close'] < data['ZLEMA'] * (1 - tolerance), 'ZLEMA_Signal'] = 1

        # Calculate the first exponential moving average (EMA1).
        ema1 = data['Close'].ewm(span=period, min_periods=period).mean()
        # Calculate the second exponential moving average (EMA2).
        ema2 = ema1.ewm(span=period, min_periods=period).mean()
        # Calculate the third exponential moving average (EMA3).
        ema3 = ema2.ewm(span=period, min_periods=period).mean()
        # Calculate the TEMA.
        tema = 3 * ema1 - 3 * ema2 + ema3

        # Convert the TEMA to a Pandas Series.
        data['TEMA'] = tema

        # Initialize column with 0s
        data['TEMA_Signal'] = 2

        # Buy signal when Close is above (1 + tolerance) * TEMA
        data.loc[data ['Close']> data ['TEMA']* (1 + tolerance), 'TEMA_Signal']= 3

        # Sell signal when Close is below (1 - tolerance) * TEMA
        data.loc[data ['Close']< data ['TEMA']*(1-tolerance), 'TEMA_Signal']= 1

        # Calculate RSI and its signal
        delta = data['Close'].diff()
        up, down = delta.copy(), delta.copy()

        up[up < 0] = 0
        down[down > 0] = 0

        average_gain = up.rolling(window=rsi_period).mean()
        average_loss = abs(down.rolling(window=rsi_period).mean())

        rs = average_gain / average_loss

        data['RSI'] =100 - (100 / (1 + rs))
        data["RSI_Signal"] =2
        data.loc[data.RSI < 30,"RSI_Signal"] =3
        data.loc[data.RSI > 70,"RSI_Signal"] =1

        # # Calculate MACD Line: (12-day EMA - 26-day EMA)
        EMA_short = data['Close'].ewm(span=short_ema_period).mean()
        EMA_long = data['Close'].ewm(span=long_ema_period).mean()
        data['MACD_Line'] = EMA_short - EMA_long

        # Calculate Signal Line: a n-day MA of MACD Line
        data['Signal_Line'] = data["MACD_Line"].ewm(span=signal_period).mean()

        # Generate MACD signals based on crossovers
        data["MACD_Signal"] = 2 # Neutral
        data.loc[(data.MACD_Line > data.Signal_Line) & (data.MACD_Line.shift() < data.Signal_Line.shift()),"MACD_Signal"] = 3 # Buy
        data.loc[(data.MACD_Line < data.Signal_Line) & (data.MACD_Line.shift() > data.Signal_Line.shift()),"MACD_Signal"] = 1 # Sell

        # Generate EMA signals based on crossovers
        data["EMA_Short"]=EMA_short
        data["EMA_Long"]=EMA_long

        # Define a small percentage as a threshold
        ema_diff_threshold = 0.01 # 1% difference

        # Calculate absolute percent difference between short and long EMA
        data['EMA_Diff'] = (EMA_short - EMA_long) / EMA_long

        data["EMA_Signal"] = np.where((data["EMA_Short"] > data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 3, 
                             np.where((data["EMA_Short"] < data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 1, 2))

        # Calculate Volatility as rolling standard deviation of log returns
        data["Log_Return"] =(np.log(data.Close).diff())
        data["Volatility"] =(data.Log_Return.rolling(window=vol_period, center=False).std())

        # Generate Volatility signals based on high or low volatility
        high_vol_threshold = data["Volatility"].quantile(0.85)
        low_vol_threshold = data["Volatility"].quantile(0.15)
        
        data['Volatility_Signal'] = 2
        data.loc[data.Volatility > high_vol_threshold, 'Volatility_Signal'] = 3
        data.loc[data.Volatility < low_vol_threshold, 'Volatility_Signal'] = 1

        # Calculate Bollinger Bands
        data["Middle_Band"] = data["Close"].rolling(window=bband_window_size, center=False).mean()
        data["Std_Dev"] = data["Close"].rolling(window=bband_window_size, center=False).std()
        data["Upper_Band"] = data["Middle_Band"] + data["Std_Dev"] * 2
        data["Lower_Band"] = data["Middle_Band"] - data["Std_Dev"] * 2

        # Identify shifting points
        data["BBand_Signal"] = 2  # neutral

        # Create a copy of the "Close" column
        close_shifted = data["Close"].copy()

        # Check for crossing Upper Band
        cross_upper_band = (close_shifted.shift(1) <= data["Upper_Band"].shift(1)) & (data["Close"] > data["Upper_Band"])
        data.loc[cross_upper_band, "BBand_Signal"] = 3  # buy

        # Check for crossing Lower Band
        cross_lower_band = (close_shifted.shift(1) >= data["Lower_Band"].shift(1)) & (data["Close"] < data["Lower_Band"])
        data.loc[cross_lower_band, "BBand_Signal"] = 1  # sell

        # Drop unnecessary columns
        data = data.drop(columns=["Middle_Band", "Std_Dev", 'Open', 'Low', 'High', 'Volume'])

    else:
        print("'Close' column is not present in the input dataframe.")

    return data


def fetch_valid_data(symbol_list_valid, start_date_valid, interval):    
   
    # Initialize an empty DataFrame
    all_data_valid = []

    today = datetime.datetime.now().strftime('%Y-%m-%d')

    for symbol in symbol_list_valid:
        # Download the stock price data
        data = yf.download(symbol, start=start_date_valid, end=today, interval=interval)
        data = data.copy()

        # Convert the index to datetime
        data.index = pd.to_datetime(data.index)
        # # print(data.index[-1])

        if interval.endswith('mo'):
            # Check if the last data point is from the current month and year
            if data.index[-1].month == today.month and data.index[-1].year == today.year:
                # Count the number of days in the current month up to today
                days_this_month_so_far = today.day

            # If the number of days in the last month is less than 20, drop the last month's data
            if days_this_month_so_far < 20:
                data = data.iloc[:-1].copy()

        elif interval.endswith('wk'):
            weekdays_till_today = (today.weekday() + 1) % 7
            # If the number of days in the last week is less than 3, drop the last week's data
            if weekdays_till_today < 3:  # 0: Monday, ..., 4: Friday
                data = data.iloc[:-1].copy()

        data.dropna(inplace=True)
        data = data[data['Volume'] != 0].copy()
        
        # Check if 'Adj Close' is in columns and drop it
        if 'Adj Close' in data.columns:
            data = data.drop('Adj Close', axis=1).copy()            
                    # Add a column to identify the symbol
        data['Symbol'] = symbol
        
                # Append the data to the all_data DataFrame
        all_data_valid.append(data)
        
    # Concatenate all_data into a single DataFrame 
    all_data_valid_df = pd.concat(all_data_valid)
    
    # # Reset the index of the final DataFrame
    # all_data_df.reset_index(inplace=True)
            
    return all_data_valid_df

def fetch_train_data(symbol_list_train, start_date, end_date, interval):    
   
    # Initialize an empty DataFrame
    all_data_train = []
    
    for symbol in symbol_list_train:
    
        data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        data = data.copy()

        data.dropna(inplace=True)
        data = data[data['Volume'] != 0].copy()
        
        # Check if 'Adj Close' is in columns and drop it
        if 'Adj Close' in data.columns:
            data = data.drop('Adj Close', axis=1).copy()            
        # Add a column to identify the symbol
        data['Symbol'] = symbol
        
        # Append the data to the all_data DataFrame
        all_data_train.append(data)
        
    # Concatenate all_data into a single DataFrame 
    all_data_train_df = pd.concat(all_data_train)
               
    return all_data_train_df

def prepare_data_separate(all_data_train_df, all_data_valid_df, symbol_list_train=None, symbol_list_valid=None, seq_len=None, target_col=None, symbol=None, 
                          interval=None, scaler=StandardScaler(), forward=-1):

    combined_train_data = None
    for symbol in symbol_list_train:

        train_data = all_data_train_df[all_data_train_df['Symbol'] == symbol].copy()    
        train_data = train_data.drop('Symbol', axis=1)

        train_data = train_data.copy()
        train_data = calculate_indicators(train_data, interval=interval, seq_len=seq_len)

        # If train_data is None, it means that the symbol is not available, so skip it
        if train_data is None:
            print(f"Not enough data for {symbol}. Skipping...") 
            continue

        train_data['Target'] = train_data[target_col].shift(forward)
        train_data.dropna(inplace=True)
        train_data = train_data.drop(target_col, axis=1)

        train_data[train_data.columns] = scaler.fit_transform(train_data)
        print(f'train_data.shape: {train_data.shape}')

        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    
    combined_valid_data = None
    for symbol in symbol_list_valid:

        valid_data = all_data_valid_df[all_data_valid_df['Symbol'] == symbol].copy()   
        valid_data = valid_data.drop('Symbol', axis=1)
        valid_data = valid_data.copy()
        valid_data = calculate_indicators(valid_data, interval=interval, seq_len=seq_len)

        # If valid_data is None, it means that the symbol is not available, so skip it
        if valid_data is None:
            print(f"Not enough data for {symbol}. Skipping...") 
            continue
        
        valid_data['Target'] = valid_data[target_col].shift(forward)
        valid_data.dropna(inplace=True)
        valid_data = valid_data.drop(target_col, axis=1)
        valid_data[valid_data.columns] = scaler.fit_transform(valid_data)
        print(f'valid_data.shape: {valid_data.shape}')

        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)

    return combined_train_data, combined_valid_data, seq_len

def prepare_data_common(train_data, valid_data, seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, seq_len)
    X_valid, y_valid = create_sequences(valid_data, seq_len)

    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)

    return X_train, y_train, X_valid, y_valid

# class DecoderOnlyTransformerModel(nn.Module):
#     def __init__(self, n_features, nhead=8, nhid=64, nlayers=6, dropout=0.1,
#                  l1_regularization=0, l2_regularization=0,
#                  activation_function=torch.nn.ReLU()):
#         super(DecoderOnlyTransformerModel, self).__init__()

#         self.pos_encoder = nn.Sequential(
#             nn.Linear(n_features, nhid),
#             activation_function,
#             nn.Linear(nhid, nhid),
#             activation_function
#         )

#         decoder_layers = TransformerDecoderLayer(nhid, nhead)
#         self.transformer_decoder = TransformerDecoder(decoder_layers,nlayers)

#         self.decoder = nn.Linear(nhid,n_features)

#         self.l1_regularization = l1_regularization
#         self.l2_regularization = l2_regularization

#     def init_weights(self):
#         # initrange = 0.1
#         nn.init.xavier_uniform_(self.pos_encoder[0].weight)
#         nn.init.xavier_uniform_(self.pos_encoder[2].weight)
#         self.decoder.bias.data.zero_()
#         nn.init.xavier_uniform_(self.decoder.weight)

#     def regularization_loss(self):
#         l1_loss = 0
#         l2_loss = 0
#         for param in self.parameters():
#             l1_loss += torch.norm(param, 1)
#             l2_loss += torch.norm(param, 2) ** 2
#         return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask

#     def forward(self, x):
#         x = self.pos_encoder(x)
#         output = torch.zeros_like(x)

#         # Process in chunks
#         chunk_size = 500  # Set based on your available memory
#         for i in range(0, x.size(0), chunk_size):
#             x_chunk = x[i:i+chunk_size]
#             tgt_mask = self.generate_square_subsequent_mask(x_chunk.size(0)).to(x.device)
#             output_chunk = self.transformer_decoder(x_chunk, x_chunk, tgt_mask=tgt_mask)
#             output[i:i+chunk_size] = output_chunk

#         output = self.decoder(output)
#         return output[:, -1:, :] 
    
# class TransformerModel(nn.Module):

#     def __init__(self, n_features, nhead=8, nhid=64, nlayers=1, dropout=0.1, l1_regularization=0, l2_regularization=0, activation_function = torch.nn.LeakyReLU()):
#         super(TransformerModel, self).__init__()
#         # model architecture
#         self.pos_encoder = nn.Sequential(
#             nn.Linear(n_features, nhid),
#             activation_function,
#             nn.Linear(nhid, nhid),
#             activation_function
#         )

#         encoder_layers = TransformerEncoderLayer(nhid, nhead, nhid, dropout,)
#         # encoder_layers.self_attn.batch_first = True
#         self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
#         self.decoder = nn.Linear(nhid, n_features)
#         self.l1_regularization = l1_regularization
#         self.l2_regularization = l2_regularization
#         self.init_weights()

#     def init_weights(self):
#         # initrange = 0.1
#         nn.init.xavier_uniform_(self.pos_encoder[0].weight)
#         nn.init.xavier_uniform_(self.pos_encoder[2].weight)
#         self.decoder.bias.data.zero_()
#         nn.init.xavier_uniform_(self.decoder.weight)

#     def regularization_loss(self):
#         l1_loss = 0
#         l2_loss = 0
#         for param in self.parameters():
#             l1_loss += torch.norm(param, 1)
#             l2_loss += torch.norm(param, 2) ** 2

#         return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

#     def forward(self, src):
#         src = self.pos_encoder(src)
#         output = self.transformer_encoder(src)
#         output = self.decoder(output)
#         output = output[:, -1:, :]

#         return output

class LSTMStockPredictor(nn.Module):
    def __init__(self, n_features, nhid=64, nlayers=1, dropout=0.1, l1_regularization=0, l2_regularization=0):
        super(LSTMStockPredictor, self).__init__()
        self.nhid = nhid
        self.nlayers = nlayers
        self.l1_regularization = l1_regularization
        self.l2_regularization = l2_regularization

        # LSTM Layer
        self.lstm = nn.LSTM(n_features, nhid, nlayers, batch_first=True, dropout=dropout)

        # Output Layer
        self.fc = nn.Linear(nhid, 1)
        self.init_weights()

    def init_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)

        self.fc.bias.data.zero_()
        nn.init.xavier_uniform_(self.fc.weight)

    def regularization_loss(self):
        l1_loss = 0
        l2_loss = 0
        for param in self.parameters():
            l1_loss += torch.norm(param, 1)
            l2_loss += torch.norm(param, 2) ** 2

        return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

    def forward(self, x):
        h0 = torch.zeros(self.nlayers, x.size(0), self.nhid).to(x.device)
        c0 = torch.zeros(self.nlayers, x.size(0), self.nhid).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  
        return out


def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam, criterion=nn.MSELoss,
                l1_regularization=0, l2_regularization=0, activation_function=None,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    criterion = criterion()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False
    
    model.train()
    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)
            reg_loss = model.regularization_loss()
            total_loss = loss + reg_loss

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            total_loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(total_loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.5f}, Val Loss = {val_losses[-1]:.5f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()

def model_save(target_col=None, n_trials=1, model_save=True, save_directory=None, plot_loss=True, symbol_list_train=None, symbol_list_valid=None, start_date=None, end_date=None, start_date_valid=None, 
               interval=None, freq=None, n_predict=5, seq_len=5, n_last_sequence=1, forward=-1):

    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    all_results_params = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")

        start = timeit.default_timer()

        # seq_len = random.choice(range(3, 6))
        # nhead = random.choice(range(15, 21))
        # nhid = nhead * random.choice(range(21, 31))
        # nlayers = random.choice(range(1, 2))
        # # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        # # activation_function = random.choice([torch.nn.GELU(), torch.nn.SiLU(), torch.nn.Mish()])
        # activation_function = random.choice([torch.nn.Mish()])
        # dropout = 0.01 * random.choice(range(9, 10))
        # optimizer = random.choice([torch.optim.Adam, torch.optim.AdamW])
        # criterion = random.choice([torch.nn.SmoothL1Loss])
        # # criterion = random.choice([torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.SmoothL1Loss, torch.nn.KLDivLoss])
        # n_epochs = random.choice(range(300, 301))
        # batch_size = random.choice(range(256, 257))
        # learning_rate = 0.00001 * random.choice(range(10, 11))
        # patience = random.choice(range(30, 31))
        # min_delta = random.choice([0.0001])
        # l1_regularization = 0.0000001 * random.choice(range(1, 2))
        # l2_regularization = l1_regularization

        seq_len = random.choice(range(3, 6))
        # nhead = random.choice(range(20, 21))
        nhid = random.choice(range(1000, 2000))
        nlayers = random.choice(range(1, 2))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        # activation_function = random.choice([torch.nn.GELU(), torch.nn.SiLU()])
        # activation_function = random.choice([torch.nn.Tanh()])
        # activation_function = random.choice([torch.nn.Mish()])
        dropout = 0 * random.choice(range(9, 10))
        optimizer = random.choice([torch.optim.AdamW])
        criterion = random.choice([torch.nn.SmoothL1Loss])
        # criterion = random.choice([torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.KLDivLoss])
        n_epochs = random.choice(range(300, 501))
        batch_size = random.choice(range(256, 257))
        learning_rate = 0.00001 * random.choice(range(10, 11))
        patience = random.choice(range(30, 31))
        min_delta = 0.00001 * random.choice(range(1, 2))
        l1_regularization = 0.0000001 * random.choice(range(4, 5))
        l2_regularization = l1_regularization

        all_data_train_df = fetch_train_data(symbol_list_train=symbol_list_train, start_date=start_date, end_date=end_date, interval=interval)

        all_data_valid_df = fetch_valid_data(symbol_list_valid=symbol_list_valid, start_date_valid=start_date_valid, interval=interval)

        train_data, valid_data, seq_len = prepare_data_separate(all_data_train_df=all_data_train_df, all_data_valid_df=all_data_valid_df, symbol_list_train=symbol_list_train, 
                                                                symbol_list_valid=symbol_list_valid, interval=interval, seq_len=seq_len, target_col=target_col, forward=forward)

        # Call prepare_data_common() with test_data_unnormalized
        X_train, y_train, X_valid, y_valid = prepare_data_common(train_data=train_data, valid_data=valid_data, seq_len=seq_len)

        # input_shape = (X_train.shape[0], seq_len, X_train.shape[2])  

        # Initialize the model
        model = LSTMStockPredictor(n_features=X_train.shape[2], nhid=nhid, nlayers=nlayers, dropout=dropout, l1_regularization=l1_regularization, l2_regularization=l2_regularization)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, criterion=criterion,
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)

        # Add the results to the results dataframe
        params = {"seq_len": seq_len, "nhid": nhid, "nlayers": nlayers, 
                "optimizer": optimizer, "criterion": criterion, "dropout": dropout, "n_epochs": n_epochs,
                "batch_size": batch_size, "learning_rate": learning_rate, "patience": patience, "min_delta": min_delta,
                "l1_regularization": l1_regularization, "l2_regularization": l2_regularization,
                "n_predict": n_predict, "n_last_sequence": n_last_sequence, "forward": forward, 'interval': interval, 'frequency': freq
                }

        results_params = {**params, "trial": trial+1, "train loss": train_losses[-1], "valid loss": val_losses[-1]}

        all_results_params.append(results_params)
        all_results_params_df = pd.DataFrame(all_results_params)

        if save_directory:
            all_results_params_df.to_csv(os.path.join(save_directory, f"all_results_params_{trial}.csv"))

        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")

    return all_results_params_df